In [1]:
import pandas as pd
import OpenDartReader
from bs4 import BeautifulSoup
import re

In [2]:
api_key = '133579488f1e7f9a44707b2a7d2be9999840935a'
dart = OpenDartReader(api_key)

In [3]:
def getReportNo(corp, year, quarter):
    bgn = {1: '03-01',
           2: '06-01',
           3: '09-01',
           4: '12-01'}[quarter]
    end = {1: '07-01',
           2: '10-01',
           3: '01-01',
           4: '04-01'}[quarter]
    bgn_day = f'{str(year)}-{bgn}'
    end_year = year
    if quarter >= 3:
        end_year += 1
    end_day = f'{str(end_year)}-{end}'

    list_a = dart.list(corp=corp, start=bgn_day, end=end_day, kind='A')
    name = {1: '분기보고서',
            2: '반기보고서',
            3: '분기보고서',
            4: '사업보고서'}[quarter]
    list_a = list_a[list_a['report_nm'].str.contains(name)]
    return list_a['rcept_no'][0]

In [12]:
def getReport(report_no):
    html = dart.document(report_no)
    soup = BeautifulSoup(html, 'html5lib')
    return soup

In [5]:
def getAssetFromReport(report):
    d0310 = report.find('title', attrs={'aassocnote': "D-0-3-1-0"}).parent
    d0310_1 = d0310.find_all('table')[1]
    asset_row = d0310_1.find('td', text=re.compile('자 *본 *총 *계')).parent
    asset = asset_row.find_all('td')[1].string
    asset = int(asset.replace(',',''))
    return asset

In [6]:
def getProfitFromReport(report):
    d0310 = report.find('title', attrs={'aassocnote': "D-0-3-1-0"}).parent
    d0310_1 = d0310.find_all('table')[3]
    profit_row = d0310_1.find('td', text=re.compile('당기순이익')).parent
    profit = profit_row.find_all('td')[1].string
    profit = int(profit.replace(',',''))
    return profit

In [15]:
def getQuarterData(corp, year, quarter):
    report_no = getReportNo(corp, year, quarter)
    report = getReport(report_no)
    profit = getProfitFromReport(report)
    asset = getAssetFromReport(report)

    if quarter == 1:
        return [asset, profit]
    
    prev_q = quarter - 1
    prev_report_no = getReportNo(corp, year, prev_q)
    prev_report = getReport(prev_report_no)
    prev_profit = getProfitFromReport(prev_report)

    return [asset, profit - prev_profit]

In [16]:
getQuarterData('00980122', 2020, 3)

[15252398, 148880]